# Chinese AD9910 PCB → Arduino

### Hardware setup

Connect the PCB to the arduino as follows:

![](connections.png)
    
Since an Arduino is used as a serial-to-SPI link to interface between the computer and the AD9910, it has to be programmed appropriately. See [`SPI_test.ino`](https://github.com/js216/Arduino-codes/blob/master/SPI_test/SPI_test.ino) for example code that is assumed in the following. Other Arduino-like boards might necessitate other pin assignments; see the [official website](https://www.arduino.cc/en/reference/SPI) for details, and adjust the provided [Arduino code](https://github.com/js216/Arduino-codes/blob/master/SPI_test/SPI_test.ino) as appropriate.
    
### Software implementation

Import packages:

In [12]:
import pyvisa

Connect to the Arduino:

In [2]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())
a = rm.open_resource('ASRL13::INSTR')

('USB0::0x1AB1::0x04CE::DS1ZA204519116::INSTR', 'ASRL8::INSTR', 'ASRL13::INSTR', 'ASRL16::INSTR', 'ASRL17::INSTR')


Write and test the function that prints the contents of one register (specified in the Arduino source code which one; currently 0x02):

In [3]:
def read_one_register(arduino):
    # send the read command
    arduino.write_raw(b"1")
    
    # print Arduino's return text
    return_text = arduino.read(), arduino.read()
    for t in return_text:
        print(t, end='')
        
read_one_register(a)

Reading from register 2:   
  00011111,00111111,01000000,00000000,


Write and test a function to read all 23 registers:

In [4]:
def read_registers(arduino):
    print(arduino.query('r'), end='')
    for i in range(23):
        print(arduino.read(), end='')
        
read_registers(a)

Reading register contents:
   0: 00000000,00000000,00000000,00000000,
   1: 00000000,01000000,00001000,00100000,
   2: 00011111,00111111,01000000,00000000,
   3: 00000000,00000000,01111111,01111111,
   4: 11111111,11111111,11111111,11111111,
   5: 11101001,00100000,01100100,10001001,11000100,11100000,
   6: 10000111,00101100,00000101,10101001,10000000,00101001,
   7: 00000000,00000000,00000000,00000000,
   8: 00000000,00000000,
   9: 00000000,00000000,00000000,00000000,
   A: 00000000,00000000,00000000,00000000,
   B: 10100001,10011001,11000100,00011010,00011001,00101110,00000011,11000100,
   C: 11000100,10101001,10110000,10000101,00000011,11000000,10000011,10110000,
   D: 01001110,11010000,00000000,00011100,
   E: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
   F: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
  10: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
  11: 00000000,00000000,00000000,00000000,00

Refer to the [AD9910 datasheet](https://www.analog.com/media/en/technical-documentation/data-sheets/AD9910.pdf), page 49ff, for a description of the register contents.

Write functions to reset the AD9910, and to write register contents:

In [5]:
def reset(arduino):
    print(arduino.query('0'))
    
def write_register(arduino, reg, data_list, printing=False):
    # send the write command
    arduino.write_raw(
        b"w" \
          + bytes(bytearray([reg])) \
          + bytes(bytearray(data_list))
      )
    
    # print Arduino's return text
    return_text = arduino.read(), arduino.read(), arduino.read(), arduino.read(),  arduino.read(),  arduino.read()
    if printing:
        for t in return_text:
            print(t, end='')

Test writing to one of the registers:

In [6]:
write_register(
    a,
    0x02,
    [0b00011111,0b00111111,0b11000000,0b00000000,],
    printing=True
  )

Reading from register 2:   
  00011111,00111111,01000000,00000000,
Writing to register 2:   
  00011111,00111111,11000000,00000000,
Reading from register 2:   
  00011111,00111111,11000000,00000000,


The following writes the register values that cause the AD9910 to output an 85 MHz sine wave for a 1 GHz clock input.

Note in particular the MSB of the second least significant byte in register 0x02, "REFCLK input divider bypass", which, when set to 1, disables the internal clock divider.

![](CFR3.png)

The amplitude, phase, and frequency settings of profile 0 are written in the 0x0E register.

In [30]:
reset(a)
write_register(a, 0x00, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x01, [0b00000000,0b01000000,0b00001000,0b00100000,])
write_register(a, 0x02, [0b00011111,0b00111111,0b11000000,0b00000000,])
write_register(a, 0x03, [0b00000000,0b00000000,0b01111111,0b01111111,])
write_register(a, 0x04, [0b11111111,0b11111111,0b11111111,0b11111111,])
write_register(a, 0x05, [0b11101001,0b00100000,0b01100100,0b10001001,0b11000100,0b11100000,])
write_register(a, 0x06, [0b10000111,0b00101100,0b00000101,0b10101001,0b10000000,0b00101001,])
write_register(a, 0x07, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x08, [0b00000000,0b00000000,])
write_register(a, 0x09, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x0A, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x0B, [0b10100001,0b10011001,0b11000100,0b00011010,0b00011001,0b00101110,0b00000011,0b11000100,])
write_register(a, 0x0C, [0b11000100,0b10101001,0b10110000,0b10000101,0b00000011,0b11000000,0b10000011,0b10110000,])
write_register(a, 0x0D, [0b01001110,0b11010000,0b00000000,0b00011100,])
for reg in range(0x0E, 0x15 + 1):
    write_register(a, reg, [0b00001000, 0b10110101, 0b00000000, 0b00000000, 0b00011101, 0b00000011, 0b01101001, 0b11010000])
write_register(a, 0x16, [0b00111000,0b11010000,0b01010011,0b11000111,])

Device reset complete.



Close connection to the Arduino:

In [128]:
a.close()

VisaIOError: VI_ERROR_CLOSING_FAILED (-1073807338): The VISA driver failed to properly close the session or object reference. This might be due to an error freeing internal or OS resources, a failed network connection, or a lower-level driver or OS error.

### Parallel data input

In order to enable controlling the AD9910 via the 18-bit parallel port, the Parallel data port enable bit in the least significant byte of register `CFR2` (0x01) has to be 1:

![](CFR2.png)

In addition, a high voltage level has to be applied to the `TxENABLE` pin of the AD9910 (`TEN` on the PCB):

![](TEN.png)

In [31]:
write_register(
    a,
    0x01,
    [0b00000000,0b01000000,0b00001000,0b00110000,],
    printing=True
  )

Reading from register 1:   
  00000000,01000000,00001000,00100000,
Writing to register 1:   
  00000000,01000000,00001000,00110000,
Reading from register 1:   
  00000000,01000000,00001000,00110000,


In case the parallel port mode isn't desired, we can again disable it:

In [32]:
write_register(
    a,
    0x01,
    [0b00000000,0b01000000,0b00001000,0b00100000,],
    printing=False
  )

### Selecting output frequency

The last four bytes of registers 0E till 15 are the frequency tuning word (FTW):

![](profile0.png)

The following function calculates the correct FTW given the clock frequency and the desired output frequency:

In [33]:
def FTW(Fsysclk, Fout):
    # calculate the exact FTW
    FTW = 2**32 * (Fout / Fsysclk)
    
    # round to the nearest integer
    FTW_int = int(FTW)
    
    # convert to bytes
    FTW_bytes = FTW_int.to_bytes(4, byteorder='big')
    
    # convert to list of integers
    return [int(x) for x in FTW_bytes]

Since we might not know which profile is currently selected, we can just write the same data to all profiles to guarantee the setting will be active:

In [34]:
for reg in range(0x0E, 0x15 + 1):
    write_register(
        a,
        reg,
        [0b00001000, 0b10110101, 0b00000000, 0b00000000] + FTW(750e6, 85e6),
        printing=False
      )

### Enabling amplitude scale factor (ASF)

First, the LSB of the most significant byte of register `CFR2` (`0x01`) needs to be set to 1 (Enable amplitude scale from single tone profiles):

In [115]:
write_register(a, 0x01, [0b00000001,0b01000000,0b00001000,0b00100000,])

There is 14 bits of amplitude information. Define a function to convert from percentage to ASF:

In [ ]:
def ASF(ASF_perc):
    # round to the nearest integer
    ASF_perc_int = int(ASF_perc)
    
    # convert to bytes
    FTW_bytes = ASF_perc_int.to_bytes(2, byteorder='big')
    
    # convert to list of integers
    return [int(x) for x in FTW_bytes]

The amplitude scale factor resides in the two most significant bytes of the profile register:

In [111]:
for reg in range(0x0E, 0x15 + 1):
    write_register(
        a,
        reg,
        [0b00011000, 0b10110101, 0b00000000, 0b00000000] + FTW(750e6, 85e6),
        printing=False
      )

Now disable ASF again when not needed:

In [116]:
write_register(a, 0x01, [0b00000000,0b01000000,0b00001000,0b00100000,])